In [1]:
import numpy as np
from folktables import ACSDataSource, ACSMobility
from matplotlib import pyplot as plt
import torch
from scipy import stats
from scipy.sparse.linalg import lobpcg
from scipy.linalg import eigh, eig
import pandas as pd
from inFairness.distances import MahalanobisDistances, SquaredEuclideanDistance
from inFairness.fairalgo import SenSeI
from inFairness.auditor import SenSeIAuditor
from tqdm.auto import tqdm
# from utils import *

/n/home10/colson/.local/lib/python3.10/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


In [2]:
r = 3
p = 30
n = 100

In [3]:
def generate_synthetic_data(n, r, p, X, S=None):
    Astar = np.random.normal(0, 1, size=(p, r)) / np.sqrt(p)
    Kstar = Astar @ Astar.T

    if not S:
        Sbar = []
        for i in range(n):
            for j in range(n):
                for k in range(n):
                    if i != j and j != k and k != i:
                        Sbar.append((i, j, k))
        Sbar = np.array(Sbar)
    
        choices = np.random.choice([False, True], size=len(Sbar), replace=True, p=[0, 1])
        S = Sbar[choices, :]

    M = []
    for t in S:
        i, j, k = t
        Mt = np.outer(X[i], X[k]) + np.outer(X[k], X[i]) - np.outer(X[i], X[j]) - np.outer(X[j], X[i]) + np.outer(X[j], X[j]) - np.outer(X[k], X[k])
        M.append(Mt)
    M = np.array(M)

    probs = 1 / (1 + np.exp(-np.einsum('ijj->i', np.einsum('ijk,kl->ijl', M,  Kstar))))

    y = 2 * np.array(probs > np.random.random(np.shape(probs)), dtype=int) - 1
    
    # y = []
    # for t in S:
    #     i, j, k = t
    #     prob = 1 / (1 + np.exp(m_dist2(X[i], X[k], Kstar) - m_dist2(X[i], X[j], Kstar)))
    #     yt = np.random.choice([1, -1], size=1, p=[prob, 1 - prob])
    #     y.append(yt)
    # y = np.array(y)
    
    return M, S, y, Astar, Kstar

def initialization(n, r, p, S, X, y):
    transition_matrices = [np.zeros((n - 1, n - 1)) for _ in range(n)]
    for t in range(len(S)):
        def gap(l, i):
            if l < i:
                return l
            else:
                return l - 1
        i, j, k = S[t]
        transition_matrices[i][gap(j, i), gap(k, i)] = 1 if y[t] == -1 else 0
        transition_matrices[i][gap(k, i), gap(j, i)] = 1 if y[t] == 1 else 0

    for i in range(n):
        d = np.max(np.sum(transition_matrices[i], axis=1))
        transition_matrices[i] = transition_matrices[i] / d

        self_loops = np.diag(1 - np.sum(transition_matrices[i], axis=1))

        transition_matrices[i] += self_loops

    dists_from_i = []
    for i in range(n):
        eigenvalues, eigenvectors = eig(transition_matrices[i], left=True, right=False)
        leading_index = np.where(np.isclose(eigenvalues, 1))[0][0]
        leading_eigenvector = eigenvectors[:, leading_index].real
        dists_wo_i = np.log(leading_eigenvector)
        if not np.all(np.isfinite(dists_wo_i)):
            print(i)
        dists = np.zeros(n)
        np.put(dists, list(range(i)) + list(range(i+1, n)), dists_wo_i)
        dists_from_i.append(dists)
    D = np.stack(dists_from_i)

    J = np.identity(n) - (np.ones((n, 1)) @ np.ones((1, n))) / n
    H = - J @ D @ J / 2
    Xprime = J @ X

    XtHX = Xprime.T @ H @ Xprime / (n**2)
    Sigma = Xprime.T @ Xprime / n
    eigenvalues, eigenvectors = eigh(a=XtHX, b=Sigma)
    U = eigenvectors[:, np.argsort(eigenvalues)[-r:]]
    Lambda = np.sort(eigenvalues)[-r:]
    Ahat = U @ np.diag(np.sqrt(Lambda))
    return Ahat

def clean_data(n, p, data):
    X = data.head(n)
    X = X.loc[:, X.var(axis=0) > 0]
    X = X.sample(p, axis=1)
    X = np.array(stats.zscore(np.array(X), axis=0))
    return X

def m_dist2(x1, x2, K):
    return (x2 - x1).T @ K @ (x2 - x1)
    

In [4]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["TX"], download=True)
acs_data_adult = acs_data[acs_data["AGEP"] >= 18]


In [5]:
acs_data_cleaned = acs_data_adult.select_dtypes(include=["float64", "int64"])
acs_data_cleaned = acs_data_cleaned.loc[:, ~(acs_data_cleaned.isna().any())]
acs_data_cleaned = acs_data_cleaned.loc[:, (acs_data_cleaned.var(axis=0) > 0)]
acs_data_cleaned = acs_data_cleaned.sample(frac=1, axis=0)
print(np.shape(np.array(acs_data_cleaned)))


(206826, 219)


In [6]:
print("Generating synthetic data...")
X = clean_data(2 * n, p, acs_data_cleaned)
X_train = X[:n]
X_test = X[n:]

Generating synthetic data...


In [20]:
M, S, y, Astar, Kstar = generate_synthetic_data(n, r, p, X_train)

print(np.shape(X_train), np.shape(X_test))

(100, 30) (100, 30)


In [21]:
transition_matrices = [np.zeros((n - 1, n - 1)) for _ in range(n)]
for i, j, k in [(i, j, k) for i in range(n) for j in range(n) for k in range(n)]: # t in range(len(S)):
    if i == j or j == k or i == k:
        continue
    def gap(l, i):
        if l < i:
            return l
        else:
            return l - 1
    # i, j, k = S[t]
    d_ij = m_dist2(X_train[i], X_train[j], Kstar)
    d_ik = m_dist2(X_train[i], X_train[k], Kstar)
    transition_matrices[i][gap(j, i), gap(k, i)] = np.exp(d_ik) / (np.exp(d_ij) + np.exp(d_ik)) # 1 if y[t] == -1 else 0
    transition_matrices[i][gap(k, i), gap(j, i)] = np.exp(d_ij) / (np.exp(d_ij) + np.exp(d_ik)) # 1 if y[t] == 1 else 0

for i in range(n):
    transition_matrices[i] -= np.diag([transition_matrices[i][j,j] for j in range(n-1)])
    d = np.max(np.sum(transition_matrices[i], axis=1))
    transition_matrices[i] = transition_matrices[i] / d

    self_loops = np.diag(1 - np.sum(transition_matrices[i], axis=1))

    transition_matrices[i] += self_loops



In [22]:
eigenvalues, eigenvectors = eig(transition_matrices[0], left=True, right=False)

In [23]:
eigenvalues

array([ 0.97574999+0.j,  0.96355369+0.j,  0.94431262+0.j,  0.93821561+0.j,
        0.9311286 +0.j,  0.98787499+0.j,  1.        +0.j,  0.91551   +0.j,
        0.89998033+0.j,  0.88625268+0.j,  0.86695662+0.j,  0.85343434+0.j,
        0.80285314+0.j,  0.84184318+0.j,  0.83778728+0.j,  0.83169762+0.j,
        0.83468574+0.j,  0.75391629+0.j,  0.76281066+0.j,  0.7313554 +0.j,
        0.7074993 +0.j,  0.71399945+0.j,  0.68763393+0.j,  0.69495559+0.j,
        0.67462199+0.j,  0.65784395+0.j,  0.64207803+0.j,  0.58327903+0.j,
        0.60848943+0.j,  0.59803613+0.j,  0.61747495+0.j,  0.6204718 +0.j,
        0.5525182 +0.j,  0.54127283+0.j,  0.52873345+0.j,  0.5484894 +0.j,
        0.50279991+0.j,  0.48420155+0.j,  0.51413964+0.j,  0.47192489+0.j,
        0.43546411+0.j,  0.45085185+0.j,  0.4544044 +0.j,  0.41204836+0.j,
        0.40022729+0.j, -0.00558294+0.j, -0.00207076+0.j,  0.35997141+0.j,
        0.35531337+0.j,  0.34881041+0.j,  0.34503076+0.j,  0.30287804+0.j,
        0.2860679 +0.j,  

In [24]:
leading_index = np.where(np.isclose(eigenvalues, 1))[0][-1]
leading_index

6

In [25]:
leading_eigenvector = eigenvectors[:, leading_index]
leading_eigenvector

array([-3.58010614e-16, -4.55418312e-18,  2.44808151e-16,  1.70805792e-16,
       -1.26905316e-16, -3.57703592e-16, -8.72241119e-16, -1.43971043e-16,
        5.18330772e-17, -1.98180137e-16, -1.13612674e-16, -8.79554541e-17,
       -1.20982970e-16, -1.34662297e-16, -1.39081929e-16, -1.25959593e-16,
       -2.53624574e-16,  2.71469471e-17,  1.50454461e-16, -2.23678266e-16,
       -1.04869625e-15, -1.00400546e-15, -4.63051450e-16, -4.03122754e-16,
        1.55798652e-16, -1.07642200e-16,  3.93158755e-15, -1.21230958e-15,
       -1.00000000e+00, -2.17197292e-16, -1.68280462e-16, -1.26351324e-17,
       -4.87855406e-16,  2.60691025e-15,  2.02249791e-16,  5.01298837e-15,
        1.92624690e-16,  3.03749699e-16,  7.31882198e-16, -3.99171538e-16,
        5.40154964e-16,  1.83899955e-16,  6.60963576e-16,  5.80387048e-17,
       -1.35669645e-16, -2.55618601e-16, -1.35376477e-17, -1.65571236e-15,
        1.30170806e-16,  8.63304239e-17, -1.73832166e-16, -9.40791610e-17,
        8.49578398e-17, -

In [15]:
(leading_eigenvector @ transition_matrices[0]) / leading_eigenvector

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.00000001, 1.        , 0.99999999,
       0.99999999, 1.00000003, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.99999999,
       1.        , 1.        , 1.00000001, 1.        , 0.99999998,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.99999997, 1.00000003, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.99999998,
       1.        , 1.00000002, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.99999999, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.00000002, 1.     

In [ ]:
dists_from_i = []
for i in range(n):
    eigenvalues, eigenvectors = eig(transition_matrices[i], left=True, right=False)
    leading_index = np.where(np.isclose(eigenvalues, 1))[0][-1]
    leading_eigenvector = eigenvectors[:, leading_index].real
    dists_wo_i = np.log(leading_eigenvector)

    #truncate


    if not np.all(np.isfinite(dists_wo_i)):
        print(i)
    dists = np.zeros(n)
    np.put(dists, list(range(i)) + list(range(i+1, n)), dists_wo_i)
    dists_from_i.append(dists)
D = np.stack(dists_from_i)

In [36]:
J = np.identity(n) - (np.ones((n, 1)) @ np.ones((1, n))) / n
H = - J @ D @ J / 2
Xprime = J @ X_train

XtHX = Xprime.T @ H @ Xprime / (n**2)
Sigma = Xprime.T @ Xprime / n

In [37]:
eigenvalues, eigenvectors = eigh(a=XtHX, b=Sigma)

In [38]:
U = eigenvectors[:, np.argsort(eigenvalues)[-r:]]


In [39]:
U

array([[-0.07182495, -0.33177168,  0.18045973],
       [ 0.03667948,  0.04839002, -0.15150435],
       [ 0.01977101,  0.01559895, -0.03604452],
       [ 0.02108022, -0.0759725 ,  0.06825837],
       [ 1.46242186, -0.75165903, -1.14678456],
       [-0.18970503,  0.07577766, -0.34092317],
       [-0.11764776,  0.11308508, -0.49613449],
       [ 0.65319258, -0.50000915, -0.80838978],
       [-1.84068402,  1.04977482,  0.567658  ],
       [-0.5429946 ,  0.07045635, -0.2620751 ],
       [-0.55522734,  0.13345332, -0.57303416],
       [-0.18883854, -0.11849849, -0.36146077],
       [ 0.40535383,  0.39588348,  0.3328791 ],
       [ 0.18858526, -0.02809735,  0.19664328],
       [ 0.30995832,  0.33386107,  0.33151063],
       [ 0.07865835,  0.0598699 ,  0.09377199],
       [ 0.05396068,  0.00206116, -0.01872646],
       [ 0.05029955,  0.01772147, -0.1607985 ],
       [-0.27801994, -0.13740375, -0.25044412],
       [-0.03648266,  0.18355382,  0.07275123],
       [-0.02848243,  0.41063991, -0.044

In [40]:
Lambda = np.sort(eigenvalues)[-r:]

In [41]:
Lambda

array([0.00547309, 0.0098498 , 0.01286321])

In [42]:
Ahat = U @ np.diag(np.sqrt(Lambda))

In [43]:
Ahat

array([[-0.00531363, -0.03292707,  0.02046704],
       [ 0.00271356,  0.00480252, -0.01718303],
       [ 0.00146267,  0.00154814, -0.00408803],
       [ 0.00155952, -0.00753998,  0.0077416 ],
       [ 0.10819046, -0.07459928, -0.13006381],
       [-0.01403444,  0.00752064, -0.03866617],
       [-0.00870362,  0.01122326, -0.05626963],
       [ 0.04832341, -0.049624  , -0.0916844 ],
       [-0.13617443,  0.10418613,  0.06438155],
       [-0.04017092,  0.00699252, -0.02972353],
       [-0.04107591,  0.01324473, -0.06499129],
       [-0.01397034, -0.01176052, -0.04099546],
       [ 0.02998821,  0.03928992,  0.03775384],
       [ 0.0139516 , -0.00278855,  0.02230251],
       [ 0.02293082,  0.03313443,  0.03759864],
       [ 0.00581917,  0.00594186,  0.01063525],
       [ 0.00399203,  0.00020456, -0.00212388],
       [ 0.00372118,  0.00175879, -0.01823714],
       [-0.02056801, -0.0136368 , -0.02840439],
       [-0.002699  ,  0.01821701,  0.00825116],
       [-0.00210714,  0.04075444, -0.005